In [1]:
import numpy as np
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import pycuda.driver as cuda

In [2]:
NUMEVENTS = 50
AVENUMJETS = 10

numjets = np.random.poisson(AVENUMJETS, NUMEVENTS)
stops = np.cumsum(numjets, dtype=np.int)
starts = np.zeros_like(stops)
starts[1:] = stops[:-1]
offsets = np.zeros(len(stops)+1, dtype=np.int)
offsets[1:] = stops

In [11]:
data = np.random.randint(low=0, high=10, size=stops[-1]).astype(np.int)

In [5]:
parents = np.empty(stops[-1], dtype=np.int)
for i in range(len(offsets)-1):
    parents[offsets[i]:offsets[i+1]] = i

In [9]:
mod = SourceModule('''

__global__ void heele_max(int* arr,int* offsets,int* parents, int* num_particles)
{
    unsigned int tid = threadIdx.x + blockIdx.x*blockDim.x;
    if (tid > num_particles[0])
        return;
    
    for (int d=0; d<log2(double(num_particles[0])); d++)
    {
        if (tid >=pow(2.0,double(d)) && parents[tid]==parents[tid-int(pow(2.0, double(d)))])
        {
            arr[tid] = max(arr[tid], arr[tid-int(pow(2.0, double(d)))]);
        }
        __syncthreads();
    }
}
''')

C:\Users\Jaydeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu



In [10]:
func = mod.get_function('heele_max')

In [12]:
len_arr = gpuarray.to_gpu(np.array([len(data)]).astype(np.int))
gpu_data = gpuarray.to_gpu(data)
gpu_parents = gpuarray.to_gpu(parents)
gpu_offsets = gpuarray.to_gpu(offsets)
numthreads = 512
numblocks = int(np.ceil(stops[-1]/numthreads))
func(gpu_data, gpu_offsets, gpu_parents, len_arr, block=(numthreads,1,1), grid=(numblocks,1))

In [13]:
host_data = gpu_data.get()

In [15]:
max_arr = np.empty_like(stops)
max_arr = host_data[stops-1]
max_arr

array([9, 9, 9, 8, 9, 9, 8, 9, 9, 9, 7, 8, 9, 8, 9, 9, 9, 9, 9, 7, 9, 9, 6,
       9, 9, 8, 8, 8, 9, 8, 8, 9, 9, 9, 7, 9, 9, 8, 9, 9, 9, 8, 9, 9, 7, 9,
       6, 9, 8, 6])

In [16]:
reduce_at_offsets = np.zeros_like(offsets)
reduce_at_offsets[1:] = stops
reduce_at_offsets[-1] = reduce_at_offsets[-1]-1
np_max = np.maximum.reduceat(data, reduce_at_offsets)

In [17]:
# check. Empty array indicates success!
np.nonzero(np_max[:-1]-max_arr)

(array([], dtype=int64),)